In [ ]:
# required depdendencies - build-essential python 3.12 python3.12-dev
%pip install setuptools autodistill autodistill-grounded-sam autodistill-yolov8 roboflow

In [2]:
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill_yolov8 import YOLOv8

# define an ontology to map class names to our GroundingDINO prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
base_model = GroundedSAM(ontology=CaptionOntology({"rabbit": "rabbit"}))


trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


In [2]:

# label all images in a folder called `context_images`
base_model.label(
  input_folder="./images",
  output_folder="./trained_dataset"
)

NameError: name 'base_model' is not defined

In [3]:
from autodistill.utils import plot
import cv2
image_path = "./animals.v3i/valid/images/Image_58_jpg.rf.5db70d270195e43b09df0b4a61548df3.jpg"
pred = base_model.predict(image_path)

plot(
    image=cv2.imread(image_path),
    detections=pred,
    classes=base_model.ontology.classes
)

The `device` argument is deprecated and will be removed in v5 of Transformers.
torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
None of the inputs have requires_grad=True. Gradients will be None


TypeError: 'method' object is not subscriptable

In [ ]:
target_model = YOLOv8("yolov8n.pt")
target_model.train("./dataset/data.yaml", epochs=200)

# run inference on the new model
pred = target_model.predict("./animals.v3i/valid/images/Image_58_jpg.rf.5db70d270195e43b09df0b4a61548df3.jpg", confidence=0.5)
print(pred)

# optional: upload your model to Roboflow for deployment
from roboflow import Roboflow

rf = Roboflow(api_key="API_KEY")
project = rf.workspace().project("PROJECT_ID")
project.version(DATASET_VERSION).deploy(model_type="yolov8", model_path=f"./runs/detect/train/")